
# Conflicts and versions 

Conflicts are essential part of any distributed system. 
Conflict arise when there exist two versions of the same message both of them being valid.
As we've seen from the previous notebooks, network can be unreliable: peer might receive the message with a long delay. Moreover, this message might have multiple valid versions. 


In [1]:
# Initialize the experiment:
import networkx as nx
import p2psimpy as p2p
import warnings
warnings.filterwarnings('ignore')

# Load the previous experiment configurations
exper = p2p.BaseSimulation.load_experiment(expr_dir='crash_gossip')

Locations, topology, peer_services, serv_impl = exper


## Client generating conflicting information

Let's assign first byzantine nodes, we will assign randomly: 

In [43]:
import random
import string

from p2psimpy.messages import GossipMessage
from p2psimpy.services.message_producer import MessageProducer

from p2psimpy.consts import TEMPERED
from p2psimpy.config import Config, Func, Dist


class ConflictMessageProducer(MessageProducer):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.balance = 100 
        
        
    def _generate_tx(self):        
        msg_phash = ''.join(random.choices(string.ascii_uppercase, k=20))
        diff = random.randint(1, 9)
        data = {'hash': msg_phash, 'balance': self.balance-diff, 'diff': diff}
        
        msg_id = '_'.join((str(self.counter), str(self.peer.peer_id)))
        msg_ttl = self.init_ttl
        return GossipMessage(self.peer, msg_id, data, msg_ttl, 
            pre_task=self.pre_task, post_task=self.post_task)
    
    def produce_transaction(self):
        # generate new transaction
        msg = self._generate_tx()
        cons = list(self.peer.connections.keys())
        m_ix = len(cons) // 2
        for p in cons[:m_ix]:
            self.peer.send(p, msg)
            
        # Generate conflicting message as if previous transaction hasn't happened  
        msg = self._generate_tx()
        for p in cons[m_ix:]:
            self.peer.send(p, msg)
        
        # Store only the last version
        self.peer.store('msg_time', str(self.counter), self.peer.env.now)
        self.peer.store('msg_data', str(self.counter), msg)
        self.balance -= msg.data['diff']
        self.counter+=1

def validate_task(msg, peer):
    # time it takes to verify the signature
    crypto_verify = Dist('norm', (1, 0.2)) 
    # time to verify the message data
    msg_verify = Dist("lognorm", (0.49512563, 4.892564, 0.0425785)) 
    
    yield peer.env.timeout(crypto_verify.get() + msg_verify.get())
    if msg.data == TEMPERED or msg.data['balance'] < 0:
        # You can decide what to do in this case.
        return False
    return True


class MsgConfig(peer_services['client'].service_map['MessageProducer']):
    pre_task = Func(validate_task)
    
peer_services['client'].service_map['MessageProducer'] = MsgConfig
serv_impl['MessageProducer'] = ConflictMessageProducer

## Run simulation 

Let's see how byzantine agents together with crashing nodes affect the message dissemination. 

In [51]:
# Init Graph
sim = p2p.BaseSimulation(Locations, topology, peer_services, serv_impl)
sim.run(6_200)

## Message analysis

Let's see how this fraction of byzantine nodes affected the network. 



In [52]:
import pandas as pd

def message_data(sim, peer_id, storage_name):
    store = sim.peers[peer_id].storage[storage_name].txs
    for msg_id, tx in store.items():
        msg_num, client_id = msg_id.split('_')
        client_tx = sim.peers[int(client_id)].storage[storage_name].txs[msg_num]
        yield (int(msg_num), tx.data == client_tx.data)
        
def get_gossip_table(sim, storage_name, func):
    return pd.DataFrame({k: dict(func(sim, k, storage_name)) 
                         for k in set(sim.types_peers['peer'])}).sort_index()

    
df = get_gossip_table(sim, 'msg_data', message_data)
df

,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
1,False,True,False,False,True,False,True,True,True,False,...,True,False,False,True,False,True,True,True,False,True
2,False,True,False,False,False,True,True,False,False,False,...,False,False,False,False,False,True,True,False,False,True
3,False,True,False,False,True,True,True,True,True,True,...,True,False,False,True,False,True,True,True,False,True
4,False,True,False,True,True,True,False,True,True,False,...,True,False,False,True,True,True,True,True,False,True
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
6,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,True,False,True,True,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,True,False,True,True,True,True,True,True,False,...,True,True,False,True,True,True,True,True,False,True
9,False,True,False,True,True,True,True,True,True,False,...,True,True,False,True,True,True,True,True,False,True
10,False,True,False,False,True,False,False,False,True,False,...,True,True,False,True,False,False,True,False,False,False


In [62]:
df[df==False].count()

1     20
2      4
3     21
4     17
5      8
6     11
7      6
8     12
9      7
10    16
11     9
12     4
13     8
14     3
15    21
16     8
17    13
18    19
19     8
20    12
21    10
22     3
23     8
24    20
25     6
dtype: int64

In [61]:
sim.peers[1].storage['msg_data'].txs

{'1_26': GossipMessage:{'hash': 'NXZOXHTZHBDDIAHKCPPE', 'balance': 99, 'diff': 1},
 '2_26': GossipMessage:{'hash': 'MFGOHOTRJXJJMCFWCRJI', 'balance': 91, 'diff': 2},
 '3_26': GossipMessage:{'hash': 'LIBCOAKDKHWHTEVYLMOE', 'balance': 78, 'diff': 9},
 '4_26': GossipMessage:{'hash': 'QJUKKBQAUENOWQSBUIZY', 'balance': 79, 'diff': 6},
 '5_26': GossipMessage:{'hash': 'ADVDKSAUIRQLZDIYUVBC', 'balance': 80, 'diff': 4},
 '6_26': GossipMessage:{'hash': 'SWARBRWVNNGPLOQMOBXP', 'balance': 77, 'diff': 2},
 '7_26': GossipMessage:{'hash': 'CNSPZXINOCIBEQIXHEIC', 'balance': 63, 'diff': 7},
 '9_26': GossipMessage:{'hash': 'AAOPSLZWIOAWFZTJZOPU', 'balance': 45, 'diff': 8},
 '8_26': GossipMessage:{'hash': 'APARJXUHZIWVEHNKSKPB', 'balance': 57, 'diff': 5},
 '10_26': GossipMessage:{'hash': 'QTDOEMSRRXNEADBBONAB', 'balance': 44, 'diff': 5},
 '11_26': GossipMessage:{'hash': 'IPFVXDUHMADIJQOFLEXF', 'balance': 37, 'diff': 4},
 '12_26': GossipMessage:{'hash': 'XPDUVHASYGRKPEXOTMWA', 'balance': 30, 'diff': 5},
 

In [60]:
sim.peers[3].storage['msg_data'].txs

{'1_26': GossipMessage:{'hash': 'NXZOXHTZHBDDIAHKCPPE', 'balance': 99, 'diff': 1},
 '2_26': GossipMessage:{'hash': 'MFGOHOTRJXJJMCFWCRJI', 'balance': 91, 'diff': 2},
 '3_26': GossipMessage:{'hash': 'LIBCOAKDKHWHTEVYLMOE', 'balance': 78, 'diff': 9},
 '4_26': GossipMessage:{'hash': 'QJUKKBQAUENOWQSBUIZY', 'balance': 79, 'diff': 6},
 '5_26': GossipMessage:{'hash': 'ADVDKSAUIRQLZDIYUVBC', 'balance': 80, 'diff': 4},
 '6_26': GossipMessage:{'hash': 'SWARBRWVNNGPLOQMOBXP', 'balance': 77, 'diff': 2},
 '7_26': GossipMessage:{'hash': 'CNSPZXINOCIBEQIXHEIC', 'balance': 63, 'diff': 7},
 '8_26': GossipMessage:{'hash': 'APARJXUHZIWVEHNKSKPB', 'balance': 57, 'diff': 5},
 '9_26': GossipMessage:{'hash': 'AAOPSLZWIOAWFZTJZOPU', 'balance': 45, 'diff': 8},
 '10_26': GossipMessage:{'hash': 'QTDOEMSRRXNEADBBONAB', 'balance': 44, 'diff': 5},
 '11_26': GossipMessage:{'hash': 'IPFVXDUHMADIJQOFLEXF', 'balance': 37, 'diff': 4},
 '12_26': GossipMessage:{'hash': 'XPDUVHASYGRKPEXOTMWA', 'balance': 30, 'diff': 5},
 

Peers see different versions of the same message. This is an issue as it might violate integrity guarantees. For example, peers might have different view on the client's balance. 

How to fix this? In short - *consensus*.


# Consensus 


Consensus is a process that allows to achieve consistent view on a value (agreement). 
Some of the peers may fail or be unreliable, so consensus protocols must be fault tolerant or resilient. The peers must communicate with one another, and agree on a single consensus value.

The consensus problem is a fundamental problem in any distributed systems. One approach to generating consensus is for all processes (agents) to agree on a majority value. In this context, a majority requires at least one more than half of available votes (where each process is given a vote). However, one or more faulty processes may skew the resultant outcome such that consensus may not be reached or reached incorrectly.


## Majority voting consensus 

Typically, this consensus family chooses a leader. This leader collects votes from other peers and decides on the canonical value based on the majority. All other peer reject alternative versions. One of the famous example of this family is PBFT. 
Modern versions and variations of this family include Hyperledger Fabric, Quorum, Tendermint etc.

### Simulate leader voting collection

TBA.

### Advantages/ Disadvantages 


TBA.

## Consensus through a Lottery

Nakamoto consensus - run a lottery to choose 

